# Sheet 05

## Preamble

Autors: Marten Ringwelski, Nico Ostermann, Simon Liessem

Note that this notebook MUST be executed in order to get everything to work.
The tasks can't be run individually. 

Also eCampus does not allow for uploading nested directory structures which makes it hard to properly organize the files. The files are expected to be in the `data` directory which itself is placed next to this notebook.

If you extract the zip file we handed in everything should work just fine.

Autoformatting if `jupyter-black` is installed.

In [ ]:
try:
    import black
    import jupyter_black

    jupyter_black.load(
        lab=False,
        line_length=79,
        verbosity="DEBUG",
        target_version=black.TargetVersion.PY310,
    )
except ImportError:
    pass

Import all we weed and more.

Set seaborn default theme

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.feature_selection import f_classif, SelectKBest
import math as m
import plotly.express as px
import sklearn.manifold
import sklearn.discriminant_analysis
import scipy as sp
import scipy.sparse
import skimage

Set seaborn default theme

In [ ]:
sns.set_theme()

If needed tweak parameters of matplotlib.
Here we increase the size and dpi to bet a bigger but still high-res image.

In [ ]:
mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["figure.figsize"] = (20, 15)
%matplotlib inline

Disable future warnings as we get a lot of them and don't really care for this sheet.

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

# Exercise 1

### a)

In [ ]:
oldtimer_rgb = plt.imread("data/oldtimer.png")
oldtimer_hsv = mpl.colors.rgb_to_hsv(oldtimer_rgb)
oldtimer_grayscale = oldtimer_hsv[:, :, 2]

Show the original image for reference.

In [ ]:
plt.imshow(oldtimer_rgb)

In [ ]:
plt.imshow(
    oldtimer_grayscale,
    cmap="gray",
    vmin=0,
    vmax=1,
)

### b)

In [ ]:
oldtimer_lab = skimage.color.rgb2lab(oldtimer_rgb)

We convert to CIE XYZ color space and then use the y which corresponds to luminance.

In [ ]:
plt.imshow(oldtimer_rgb[:, :, 1], cmap="gray")

# Exercise 2

For this whole task, note that we cannot influence how the plots are displayed in the pdf.

If you run the code you can move them around.

### a)

The main insight is that w ewant to generate all convex combinations whose factors sum up to 1.

In [ ]:
def regular_sample_triangle(a, b, c, amount):
    """Returns points that are convex combinations of a, b and c.
    The number of points returned is at least `amount`.
    Additionally the convex factors have to sum up to 1."""
    # Our goal is to have all convex combinations that sum up to 1.
    # We use stars and bars to split the interval [0,1] in all possible ways.
    # We know that the amount of combinations is (n + k -1) choose k
    # where k = 2 since we draw two dividers.
    #
    # https://mathworld.wolfram.com/Multichoose.html

    # We used wolframalpha to solve the formula for n
    n = m.ceil(0.5 * (m.sqrt(8 * amount + 1) - 3))

    stars = np.linspace(0, 1, n)

    bars = np.array(np.meshgrid(np.arange(n), np.arange(n))).T.reshape(-1, 2)
    bars = bars[bars[:, 0] <= bars[:, 1]]

    values = np.array(
        [
            stars[bars[:, 0]],
            stars[bars[:, 1]] - stars[bars[:, 0]],
            np.ones(bars.shape[0]) - stars[bars[:, 1]],
        ],
    )

    pts = values.T @ np.array([a, b, c])

    return pts

Now finally sample some points.

In [ ]:
r = np.array([1, 0, 0])
g = np.array([0, 1, 0])
b = np.array([0, 0, 1])

amount = 100

triangle_rgb = regular_sample_triangle(r, g, b, amount)

### b)

Now show a plot.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.set_xlabel("red")
ax.set_ylabel("green")
ax.set_zlabel("blue")

ax.scatter(
    triangle_rgb[:, 0],
    triangle_rgb[:, 1],
    triangle_rgb[:, 2],
    c=triangle_rgb,
)

### c)

In [ ]:
rgb = np.array([r, g, b])

We start by converting the whole triangle to CIE Lab.
Then we plot using the rgb colors.

In [ ]:
cie_lab = skimage.color.rgb2lab(triangle_rgb)

In [ ]:
triangle_lab = cie_lab @ rgb

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.set_xlabel("red")
ax.set_ylabel("green")
ax.set_zlabel("blue")

ax.scatter(
    triangle_lab[:, 0],
    triangle_lab[:, 1],
    triangle_lab[:, 2],
    c=triangle_rgb,
)

Now the same for CIE XYZ.

In [ ]:
cie_xyz = skimage.color.rgb2xyz(triangle_rgb)

In [ ]:
triangle_xyz = cie_xyz @ rgb

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.set_xlabel("red")
ax.set_ylabel("green")
ax.set_zlabel("blue")

ax.scatter(
    triangle_xyz[:, 0],
    triangle_xyz[:, 1],
    triangle_xyz[:, 2],
    c=triangle_rgb,
)

### d)

In [ ]:
mds = sklearn.manifold.MDS()

In [ ]:
triangle_rgb_mds = mds.fit_transform(triangle_rgb)
triangle_lab_mds = mds.fit_transform(triangle_cie_lab)
triangle_xyz_mds = mds.fit_transform(triangle_cie_xyz)

In [ ]:
fig, axs = plt.subplots(1, 3, subplot_kw={"aspect": "equal"})
ax_rgb, ax_lab, ax_xyz = axs

ax_rgb.set_title("rgb")
ax_lab.set_title("lab,")
ax_xyz.set_title("xyz")

In [ ]:
ax_rgb.scatter(
    triangle_rgb_mds[:, 0],
    triangle_rgb_mds[:, 1],
    c=triangle_rgb,
)

ax_lab.scatter(
    triangle_lab_mds[:, 0],
    triangle_lab_mds[:, 1],
    c=triangle_rgb,
)

ax_xyz.scatter(
    triangle_xyz_mds[:, 0],
    triangle_xyz_mds[:, 1],
    c=triangle_rgb,
)

In [ ]:
fig

TODO comment on shapes